In [367]:
import glob
import os
import pandas as pd

define constants

In [368]:
keys = ["top","tok","mess","rush","sil","s_m","s_n","er","EOS",]
column_names = ["Mwfsending", "Mwfreceiving", "Mwfreturning", "Mcirctime", "top","tok","mess","rush","sil","s_m","s_n","er"]
path_to_write_data = "D:\\Git\\TokenRing\\research\\"
path_to_data = path_to_write_data+"data\\"
filename = "results.txt"

define functions

In [369]:
def createColumnNames():
    result = ""
    for column_name in column_names:
        result = result + column_name + " "
    return result

In [370]:
def processFilename(filename):
    result = ""
    for x in range(0, len(keys)-1):
        result = result + filename[filename.find(keys[x])+len(keys[x]): filename.find(keys[x+1])]+" "
    return result

In [371]:
def appendStr(appendix):
    if os.path.exists(path_to_write_data+filename):
        append_write = 'a' # append if already exists
    else:
        append_write = 'w' # make a new file if not

    file_with_results = open(path_to_write_data+filename, append_write)
    if append_write == "w":
            file_with_results.write(createColumnNames()+"\n")
    file_with_results.write(appendix+"\n")
    file_with_results.close()

In [372]:
def collectResults(filename):
    names = ['Created', 'Sent', 'Received', 'Returned']
    data = pd.read_table(filename, sep=' ', names=names)
    data = data.drop(data.index[0])
    data = data.apply(pd.to_numeric, errors='coerce')
    wait_for_sending = data["Sent"] - data["Created"]
    wait_for_receiving = data["Received"] - data["Created"]
    circulated_before_receiving = data["Received"] - data["Sent"]
    time = data["Created"] - min(data["Created"])
    wait_for_returning = data["Returned"] - data["Created"]
    circulated_before_return = data["Returned"] - data["Sent"]
    not_returned = data.loc[data["Returned"]==0].index
    circulated_before_return_good = circulated_before_return.drop(not_returned)
    wait_for_returning_good = wait_for_returning.drop(not_returned)
    time_good = time.drop(not_returned)
    result = str(wait_for_sending.median())
    result += " "
    result += str(wait_for_receiving.median())
    result += " "
    result += str(wait_for_returning_good.median())
    result += " "
    result += str(circulated_before_return_good.median())
    result += " "
    return result

In [373]:
def takeDataFromFile(elem):
    name = elem.replace("D:\\Git\\TokenRing\\research\\data\\", "")
    result = collectResults(elem)+processFilename(name)
    print result
    os.rename(elem, path_to_data+"used\\"+name)
    return result

create dataset with all measurements

In [374]:
t = glob.glob(path_to_data+"*.txt")

In [375]:
for elem in t:
    appendStr(takeDataFromFile(elem))

5741153.5 7667738.0 9975399.0 4287347.0 27 12 1000 false true 10 10 false 
46556624.5 49976952.5 52055030.0 5321700.0 29 12 1000 false true 10 10 false 
1752779.0 3700559.0 5555300.0 3802298.0 25 14 1000 false true 10 10 false 
2908059.5 5430133.0 7418967.0 4389533.0 27 14 1000 false true 10 10 false 
225344.5 977236.5 1721543.0 1421456.0 15 15 1000 false true 10 10 false 
14978494.5 17677050.0 19803767.5 5064227.5 29 14 1000 false true 10 10 false 
524986.0 1466524.5 2373034.0 1789369.0 17 15 1000 false true 10 10 false 
502898.0 1751887.0 2778654.5 2161299.5 19 15 1000 false true 10 10 false 
916550.0 2496416.0 3801629.0 2780885.0 21 15 1000 false true 10 10 false 
1136093.0 2958037.0 4503989.5 3194091.5 23 15 1000 false true 10 10 false 
2143450.0 4163519.0 5972745.0 3769278.0 25 15 1000 false true 10 10 false 
2640770.0 5065120.0 7131150.0 4371684.0 27 15 1000 false true 10 10 false 
4093684.0 6765244.0 9175761.0 4957134.0 29 15 1000 false true 10 10 false 
295402.0 1278886.0 21262